In [55]:
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict

import time
import numpy as np
from lsh import clsh, jlsh, generateSamples, findNeighborsBrute, recall

# open docs file and read its lines
with open("train.dat", "r") as fh:
    lines = fh.readlines()

i=0
for l in lines:
    s=l.split()
    #print s
    train_labels[i]=s[0]
    i=i+1



train_documents=  [l.split()[1:] for l in lines]
  
#print train_labels[0:10]
#print train_documents[0]


def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]


docs1 = filterLen(train_documents, 3)


#print(len(train_documents[0]), train_documents[0][:20])
#print(len(docs1[0]), docs1[0][:20])


def plotWf(docs, plot=True, logscale=True):
    r"""Get collection-wide word frequencies and optionally plot them."""
    words = defaultdict(int)
    for d in docs:
        for w in d:
            words[w] += 1
    if plot is True:
        plt.plot(sorted(words.values(), reverse=True))
        plt.xlabel('word')
        plt.ylabel('frequency')
        if logscale is True:
            plt.yscale('log')
            plt.ylabel('log(frequency)')
        plt.title('Corpus-wide word frequency distribution')
        plt.show()
    return words

def plotDf(docs, plot=True, logscale=False):
    r"""Get collection-wide word frequencies and optionally plot them."""
    # document word frequency
    df = defaultdict(int)
    for d in docs:
        for w in set(d):
            df[w] += 1
    if plot is True:
        plt.plot(sorted(df.values(), reverse=True))
        plt.xlabel('word')
        plt.ylabel('frequency')
        if logscale is True:
            plt.yscale('log')
            plt.ylabel('log(frequency)')
        plt.title('Corpus-wide document-word frequency distribution')
        plt.show()
    return df



from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [56]:
mat = build_matrix(train_documents)
csr_info(mat)


 [nrows 25000, ncols 283110, nnz 3784021]


In [58]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
mat2 = csr_idf(mat, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)
print("mat1:", mat[15,:20].todense(), "\n")
print("mat2:", mat2[15,:20].todense(), "\n")
print("mat3:", mat3[15,:20].todense())

('mat1:', matrix([[ 0.,  3.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
          2.,  0.,  2.,  0.,  0.,  2.,  0.]]), '\n')
('mat2:', matrix([[ 0.        ,  0.12071649,  0.        ,  0.38748767,  0.        ,
          0.        ,  0.1195496 ,  0.        ,  0.        ,  0.        ,
          0.        ,  1.90059267,  0.        ,  0.37690462,  0.        ,
          0.66826196,  0.        ,  0.        ,  1.03610334,  0.        ]]), '\n')
('mat3:', matrix([[ 0.        ,  0.00215428,  0.        ,  0.00691503,  0.        ,
          0.        ,  0.00213346,  0.        ,  0.        ,  0.        ,
          0.        ,  0.03391759,  0.        ,  0.00672616,  0.        ,
          0.01192567,  0.        ,  0.        ,  0.01849009,  0.        ]]))


In [64]:
from scipy.sparse.linalg import norm
i = 15  # one row
j = 90  # another row
# compare cosine similarity of rows from mat2 vs. mat3
dp2 = mat2[i].dot(mat2[j].T).todense().item()  # the dot-product between the sparse vectors in mat2
print "dot-product in mat2: ", dp2
print "norms in mat2: ", norm(mat2[i]), norm(mat2[j])
print "********** cosine in mat2: ", dp2 / ( norm(mat2[i]) * norm(mat2[j]) )
dp3 = mat3[i].dot(mat3[j].T).todense().item()  # the dot-product between the sparse vectors in mat3
print "dot-product in mat3: ", dp3
print "norms in mat3: ", norm(mat3[i]), norm(mat3[j])
print " ********** cosine in mat3: ", dp3 / ( norm(mat3[i]) * norm(mat3[j]) )

dot-product in mat2:  18.8729419634
norms in mat2:  56.0356024054 56.1620050738
********** cosine in mat2:  0.00599698489363
dot-product in mat3:  0.00599698489363
norms in mat3:  1.0 1.0
 ********** cosine in mat3:  0.00599698489363


In [65]:
import time
import numpy as np
from lsh import clsh, jlsh, generateSamples, findNeighborsBrute, recall


